In [ ]:
!pip install kaggle

In [ ]:
!pip3 install wandb

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [8]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

 98% 325M/330M [00:02<00:00, 141MB/s]
100% 330M/330M [00:02<00:00, 147MB/s]


In [9]:
!unzip -qq face-mask-12k-images-dataset.zip

In [10]:
import wandb
import numpy as np
import cv2
import tensorflow as tf
from wandb.keras import WandbCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Flatten, BatchNormalization, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
wandb.init(project='face-mask-recognition')

In [13]:
batch_size = 256
lr = 0.001
epoch = 40
width = height = 64

In [14]:
data_generator = ImageDataGenerator(
                                  rescale = 1 / 255,
                                  horizontal_flip = True
                                  )

train_data = data_generator.flow_from_directory("Face Mask Dataset/Train/",
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True
                                                    )

val_data = data_generator.flow_from_directory("Face Mask Dataset/Validation/",
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True
                                                    )

test_data = data_generator.flow_from_directory("Face Mask Dataset/Test/",
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True
                                                    )

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([

    Conv2D(96, (11, 11), activation= 'relu', input_shape = (width, height, 3)),
    BatchNormalization(),
    MaxPooling2D(2),


    Conv2D(256, (11, 11), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),


    Conv2D(384, (3, 3), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),


    Conv2D(384, (3, 3), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),


    Conv2D(256, (3, 3), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),
    MaxPooling2D(2, strides= (2, 2)),


    Flatten(),


    Dense(4096, activation= 'relu'),
    Dropout(0.5),


    Dense(4096, activation= 'relu'),
    Dropout(0.5),


    Dense(1, activation='sigmoid')                          
])

In [15]:
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(width, height, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'max'
    )

94683136/94668760 [==============================] - 1s 0us/step


In [16]:
for layer in base_model.layers[:-8]:
  layer.trainable = False

In [17]:
model = tf.keras.models.Sequential([
    base_model,
    Dense(2, activation='softmax')                          
])

In [18]:
model.compile(optimizer = tf.keras.optimizers.SGD(),
              loss = tf.keras.losses.categorical_crossentropy,
              metrics='accuracy')

In [19]:
model.fit(train_data,
          steps_per_epoch = train_data.samples / batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples / batch_size,
          epochs = 10,
          callbacks = [WandbCallback()]
)

Epoch 1/10
39/39 [==============================] - 57s 490ms/step - loss: 0.1109 - accuracy: 0.9633 - val_loss: 0.0607 - val_accuracy: 0.9800


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/10
39/39 [==============================] - 18s 447ms/step - loss: 0.0247 - accuracy: 0.9922 - val_loss: 0.0413 - val_accuracy: 0.9900
Epoch 3/10
39/39 [==============================] - 17s 436ms/step - loss: 0.0175 - accuracy: 0.9960 - val_loss: 0.0305 - val_accuracy: 0.9925
Epoch 4/10
39/39 [==============================] - 17s 437ms/step - loss: 0.0095 - accuracy: 0.9980 - val_loss: 0.0369 - val_accuracy: 0.9912
Epoch 5/10
39/39 [==============================] - 17s 429ms/step - loss: 0.0062 - accuracy: 0.9983 - val_loss: 0.0380 - val_accuracy: 0.9900
Epoch 6/10
39/39 [==============================] - 17s 429ms/step - loss: 0.0050 - accuracy: 0.9990 - val_loss: 0.0346 - val_accuracy: 0.9925
Epoch 7/10
39/39 [==============================] - 17s 428ms/step - loss: 0.0031 - accuracy: 0.9995 - val_loss: 0.0396 - val_accuracy: 0.9912
Epoch 8/10
39/39 [==============================] - 17s 433ms/step - loss: 0.0025 - accuracy: 0.9999 - val_loss: 0.0347 - val_accuracy: 0.9925

In [20]:
model.save('/content/drive/MyDrive/facemask_model_resnet.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [21]:
Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis= 1)

print(confusion_matrix(test_data.classes, y_pred))
targets = set(test_data.class_indices.keys())
print('classification_report:')
print(classification_report(test_data.classes, y_pred, target_names= targets))

[[229 254]
 [257 252]]
classification_report:
              precision    recall  f1-score   support

 WithoutMask       0.47      0.47      0.47       483
    WithMask       0.50      0.50      0.50       509

    accuracy                           0.48       992
   macro avg       0.48      0.48      0.48       992
weighted avg       0.48      0.48      0.48       992

